In [ ]:
!pip3 install  -r requirements.txt

In [3]:
from dotenv import dotenv_values
import openai, os
import numpy as np
from numpy.linalg import norm

secrets= dotenv_values(".env")

In [4]:
os.environ['OPENAI_API_KEY'] = secrets['OPENAI_API_KEY']

In [27]:
from unstructured.partition.pdf import partition_pdf

path = "data/"

raw_pdf_elements = partition_pdf(
    filename=path + "RAG_benchmark.pdf",
    extract_images_in_pdf=True,
    infer_table_structure=True,
    chunking_strategy="by_title",
    max_characters=1000, 
    new_after_n_chars=800, 
    combine_text_under_n_chars=500,
    image_output_dir_path=path + 'images/'
)

In [28]:
raw_pdf_elements[1].text 

'Abstract'

In [22]:
raw_pdf_elements

In [57]:
table_elements = []
text_elements = []
for element in raw_pdf_elements:
    if element.category == "Table":
        table_elements.append(element.text)
    elif element.category == "CompositeElement":
        text_elements.append(element.text)


table_elements = list(filter(None, table_elements))
text_elements = list(filter(None, text_elements))
print(len(table_elements))
print(len(text_elements))

12
67


## Summarize

In [59]:
from langchain.chains import LLMChain
from langchain.output_parsers import NumberedListOutputParser
from langchain.chat_models import ChatOpenAI

prompt = """You are an AI PhD specialized in AI and RAG, you are tasked with summarizing tables and text. 
Give a concise summary of the table or text.

Table or text chunk: {element}
""" 
model_name='gpt-4' #"text-embedding-3-small" 
model = ChatOpenAI(temperature=0, model_name=model_name)
summarize_chain = LLMChain.from_string(
    llm=model,
    template=prompt
)

In [61]:
summarize_chain.batch(table_elements)

[{'element': 'Benchmark Web KG Mock Dynamic Torso and Beyond retrieval search API question tail facts Wikipedia QALD-10 [24] x v x x x x MS MARCO [3] v x x not explicitly not explicitly v Natural Questions [12] v x x not explicitly not explicitly x RGB [5] v x x x x v FreshLLM [25] x x x v x v CRAG v v v v v v',
  'text': "The table provides a comparison of different AI systems in terms of their capabilities. The systems compared are QALD-10 [24], MS MARCO [3], Natural Questions [12], RGB [5], FreshLLM [25], and CRAG. The features compared include Benchmark Web, KG Mock, Dynamic Torso, Beyond retrieval, search API, question tail, facts, and Wikipedia. CRAG is the only system that supports all the features. MS MARCO [3] and Natural Questions [12] do not explicitly support 'Beyond retrieval' and 'search API'. FreshLLM [25] supports 'Dynamic Torso' and 'facts' but not the other features. RGB [5] does not support 'KG Mock' and 'Dynamic Torso'. QALD-10 [24] only supports 'Benchmark Web'."},